In [ ]:
Steps:

1. Read the PDF file - Done

2. Convert the PDF file into small chunk - Done 

3. Create a simplified prompt using llamaIndex - This we will use multi prompt

4. Splitting the document and create english embeddings 

5. Loading the data into Chroma DB - Done

6. Summarization of the whole document - Matching with similarity index of 5. - Done


In [ ]:
pip install llama_index llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface llama-index-vector-stores-chroma llama-index-llms-mistralai qdrant-client mistralai

In [2]:
pip install pypdf2

Note: you may need to restart the kernel to use updated packages.


In [3]:
import PyPDF2

# Define the PDF and text file paths
pdf_file = "./bookpdf/AtomicHabits.pdf"
text_file = "./atomichabit/extracted_atomichabit.txt"
text = ""

# Open the PDF file
with open(pdf_file, 'rb') as f:
  pdf_reader = PyPDF2.PdfReader(f)
    # Extract text and store it in a variable
  for page in pdf_reader.pages:
    text += page.extract_text()

# Open the text file in write mode and write the extracted text
with open(text_file, "w") as f:
  f.write(text)

print(f"Text extracted from {pdf_file} and saved to {text_file}")

Text extracted from ./bookpdf/AtomicHabits.pdf and saved to ./atomichabit/extracted_atomichabit.txt


In [4]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

collection_name = "bookparser"
chroma_client = chromadb.PersistentClient()

# Hypothetical method to list all collections. The actual method name may vary.
collections = chroma_client.list_collections()

# Check if the collection exists by looking through the list of collections
if any(collection.name == collection_name for collection in collections):
    # Proceed to delete the collection
    try:
        chroma_client.delete_collection(collection_name)
        print(f"Collection '{collection_name}' has been deleted.")
    except Exception as e:
        print(f"Failed to delete collection '{collection_name}': {e}")

chroma_collection = chroma_client.create_collection(collection_name)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Collection 'bookparser' has been deleted.


In [5]:
# Global settings
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
# Local settings
from llama_index.core.node_parser import SentenceSplitter

Settings.chunk_size = 512
documents = SimpleDirectoryReader("atomichabit").load_data()

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context,  transformations=[SentenceSplitter(chunk_size=512)]
)
query_engine = index.as_query_engine()
response = query_engine.query("What did the author talk about?")
print(response)

The author discussed his journey from personal experiments with habits to sharing them publicly through articles on his website, which eventually led to a significant increase in email subscribers. This success in writing about habits led to speaking opportunities at companies, keynote speeches at conferences, and having articles published in major publications, reaching a wide audience.


In [12]:
pip install llama-index

Note: you may need to restart the kernel to use updated packages.


In [17]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
import json 
import random

def chat_with_query_engine(query):
    """Get the information in summarized form based on the query asked"""
    print(f"Calling local chat_with_query_engine for {query}")
    return json.dumps({"query": query, "temperature": random.randint(20,50)})

llm = OpenAI(model="gpt-3.5-turbo-1106")
tool = FunctionTool.from_defaults(fn=chat_with_query_engine)
agent = OpenAIAgent.from_tools([tool], llm=llm, verbose=True)
response = agent.chat(
    "What did the author talk about?"
)

print(response)


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x103709910>>
Traceback (most recent call last):
  File "/Users/rajeevsingla/Library/Python/3.11/lib/python/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


Added user message to memory: What did the author talk about?
=== Calling Function ===
Calling function: chat_with_query_engine with args: {"query":"summary"}
Calling local chat_with_query_engine for summary
Got output: {"query": "summary", "temperature": 30}

The author discussed various topics related to the subject matter. If you have specific questions or topics in mind, feel free to ask!
